# **Finetuning de LLMs abertas**

Uma equipe de análise de dados de uma empresa precisa consultar informações do banco de dados com frequência para gerar relatórios e insights. Porém, nem todos os analistas têm conhecimentos avançados em SQL, o que gera uma dependência dos desenvolvedores para escrever essas consultas.

Além disso, o banco de dados possui informações de clientes que são sigilosas e a empresa não gostaria de utilizar grandes modelos de empresas que poderiam coletar dados e vazar informações.

Nosso papel nesse projeto é realizar o fine-tuning de um modelo de LLM aberta que converta comandos em linguagem natural para SQL, permitindo que os analistas façam suas consultas localmente e obtenham as informações que precisam sem precisar de suporte contínuo dos desenvolvedores e ao mesmo tempo não compartilhem os dados com APIs externas.

# **Gerando respostas com uma LLM**

### **Carregando o modelo Llama**

Para que seja possível utilizar uma LLM localmente, precisamos carregar um modelo mais leve, caso contrário o computador não conseguirá processar os resultados.

O [Unsloth](https://unsloth.ai/) fornece LLMs de código aberto e opções quantizadas dos modelos que reduz a memória necessária para o carregamento e melhora a velocidade de processamento:

- [Modelos de código aberto](https://huggingface.co/unsloth)

Vamos instalar a biblioteca Unsloth e pré-requisitos para carregar um modelo Llama. Precisamos utilizar uma GPU para utilização da biblioteca, portanto vamos usar a GPU T4 do Google Colab.

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

- Link git Unsloth: `'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'`



In [ ]:
# !pip install --upgrade --no-deps 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-d9znb4i_/unsloth_3e1c818559194e81b1ddeb411d577a78
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-d9znb4i_/unsloth_3e1c818559194e81b1ddeb411d577a78
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.10-py3-none-any.whl size=166794 sha256=543c7764bc1e8cda0bd7995840953354aed23c70ce3dbd75de12f9d4f464c981
  Stored in directory: /tmp/pip-ephem-wheel-cache-lkjboi0_/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2024.11.11
    Uninstalling unsloth-2024.11.11:
      Successfully uninstalled unsloth-2024.11

In [ ]:
# !pip install  --no-deps torch xformers trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Vamos utilizar o modelo LLama 3.1 com 8 bilhões de parâmetros. É um modelo de código aberto, por conta disso não precisamos de acessar nenhuma API, nem pagar nenhum valor para utilizar:

- [Llama 3.1-8B Hugging Face](https://huggingface.co/unsloth/Meta-Llama-3.1-8B)

In [ ]:
checkpoint_modelo = 'unsloth/Meta-Llama-3.1-8B'

No momento de fazer o carregamento do modelo, vamos utilizar parâmetros para utilizar menos memória.

- dtype: None para detecção automática, Float16 para Tesla T4, V100, Bfloat16 para Ampere+
- load_in_4bit: Utiliza menos memória ao reduzir a quantidade de bits de informação. Menos preciso.

In [ ]:
modelo, tokenizador = FastLanguageModel.from_pretrained(
    model_name = checkpoint_modelo,
    max_seq_length=2048,
    dtype = None,
    load_in_4bit=True
)

In [ ]:
modelo

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): L

In [ ]:
tokenizador

PreTrainedTokenizerFast(name_or_path='unsloth/meta-llama-3.1-8b-bnb-4bit', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	12

### **Gerando consultas com o modelo**

Com o modelo carregado, podemos utilizá-lo para gerar texto a partir de um prompt. Vamos testar o modelo para que ele realize a tarefa de gerar querys SQL a partir de uma pergunta.

In [ ]:
prompt = 'Me dê uma query SQL para saber quantas pessoas tem mais de 56 anos.'

In [ ]:
prompt_tokenizado = tokenizador([prompt], return_tensors='pt').to('cuda')

In [ ]:
prompt_tokenizado

{'input_ids': tensor([[128000,   7979,    294,   5615,  10832,   3319,   8029,   3429,  42104,
          10484,    300,  47062,   1592,  10071,    409,    220,   3487,  38101,
             13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
from transformers import TextStreamer

In [ ]:
FastLanguageModel.for_inference(modelo)
streamer_texto = TextStreamer(tokenizador)

_ = modelo.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens = 128)

<|begin_of_text|>Me dê uma query SQL para saber quantas pessoas tem mais de 56 anos. A query deve ser escrita utilizando a função COUNT() e o operador WHERE.
```
SELECT COUNT(*) FROM pessoa WHERE idade > 56;
```
Me dê uma query SQL para saber quantas pessoas tem mais de 56 anos. A query deve ser escrita utilizando a função COUNT() e o operador WHERE.
```
SELECT COUNT(*) FROM pessoa WHERE idade > 56;
```<|end_of_text|>


O modelo pode gerar um texto inesperado e alterar o objetivo da pergunta inicial, tirando todo o propósito de utilizar o modelo para gerar a query correta.

# **Preparando textos para o finetuning**

### **Carregando os dados**

Vamos carregar dados com perguntas e respostas no formato que queremos que o modelo responda. Esse é um formato padrão para realizar o finetuning de modelos de geração de texto.

- [Base de dados text-to-SQL (Hugging Face)](https://huggingface.co/datasets/emdemor/sql-create-context-pt)

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('emdemor/sql-create-context-pt', split = 'train')

In [ ]:
dataset

Dataset({
    features: ['pergunta', 'contexto', 'resposta'],
    num_rows: 78577
})

In [ ]:
dataset.to_pandas()

,pergunta,contexto,resposta
0,Quantos chefes de departamento têm mais de 56 ...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56
1,"Indicar o nome, estado de nascimento e idade d...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B..."
2,"Indique o ano de criação, o nome e o orçamento...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM..."
3,Qual é o orçamento máximo e mínimo dos departa...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,Qual é o número médio de empregados dos depart...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...
...,...,...,...
78572,A que horas foi o jogo com a pontuação de 3-2?,"CREATE TABLE table_name_35 (time VARCHAR, scor...","SELECT time FROM table_name_35 WHERE score = ""..."
78573,Em que terreno a equipa jogou contra o Aston V...,"CREATE TABLE table_name_83 (ground VARCHAR, op...",SELECT ground FROM table_name_83 WHERE opponen...
78574,Que tipo de competição foi no San Siro às 18h3...,CREATE TABLE table_name_60 (competition VARCHA...,SELECT competition FROM table_name_60 WHERE gr...
78575,Qual é o número total de decílios para a local...,"CREATE TABLE table_name_34 (decile VARCHAR, na...",SELECT COUNT(decile) FROM table_name_34 WHERE ...


### **Formatando o prompt**

Além dos dados, podemos formatar um prompt com as instruções padronizadas. Isso facilita o entendimento do modelo e direciona para que responda no formato adequado. Modelos de prompt:

- [Alpaca](https://ollama.com/mannix/llamax3-8b-alpaca/blobs/ec8a066fd4f1)
- [Llama 3](https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-3/)
- [Mistral](https://ollama.com/library/mistral/blobs/491dfa501e59)
- [Gemma 2](https://ollama.com/library/gemma2/blobs/109037bec39c)

In [ ]:
def gerar_prompt_sql(contexto, pergunta, resposta = ''):
    return f'''Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

Você deve gerar a consulta SQL que responde à pergunta.

### Instruction:
Contexto: {contexto}

### Input:
Pergunta: {pergunta}

### Response:
Resposta: {resposta}
'''

In [ ]:
dataset[0]

{'pergunta': 'Quantos chefes de departamento têm mais de 56 anos ?',
 'contexto': 'CREATE TABLE head (age INTEGER)',
 'resposta': 'SELECT COUNT(*) FROM head WHERE age > 56'}

In [ ]:
print(gerar_prompt_sql(dataset[0]['contexto'], dataset[0]['pergunta'], dataset[0]['resposta']))

Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

Você deve gerar a consulta SQL que responde à pergunta.

### Instruction:
Contexto: CREATE TABLE head (age INTEGER)

### Input:
Pergunta: Quantos chefes de departamento têm mais de 56 anos ?

### Response:
Resposta: SELECT COUNT(*) FROM head WHERE age > 56



### **Transformando todos os dados**

Depois de realizar a formatação dos dados a partir do prompt, vamos tokenizar os dados para que o modelo consiga compreender as informações.

Vamos utilizar um [*EOS TOKEN*(*end-of-sequence*)](https://huggingface.co/docs/transformers/main/llm_tutorial) para delimitar quando termina uma sentença para indicar ao modelo quando termina o prompt.



In [ ]:
EOS_TOKEN = tokenizador.eos_token

In [ ]:
EOS_TOKEN

'<|end_of_text|>'

In [ ]:
def formatar_prompts(dados):
    contextos = dados['contexto']
    perguntas = dados['pergunta']
    respostas = dados['resposta']
    textos = []
    for contexto, pergunta, resposta in zip(contextos, perguntas, respostas):
        texto = gerar_prompt_sql(contexto, pergunta, resposta) + EOS_TOKEN
        textos.append(texto)
    return {'texto': textos,}

In [ ]:
dataset = dataset.map(formatar_prompts, batched = True)

In [ ]:
dataset.to_pandas()

,pergunta,contexto,resposta,texto
0,Quantos chefes de departamento têm mais de 56 ...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56,Você é um modelo poderoso de texto-para-SQL. S...
1,"Indicar o nome, estado de nascimento e idade d...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B...",Você é um modelo poderoso de texto-para-SQL. S...
2,"Indique o ano de criação, o nome e o orçamento...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM...",Você é um modelo poderoso de texto-para-SQL. S...
3,Qual é o orçamento máximo e mínimo dos departa...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_...",Você é um modelo poderoso de texto-para-SQL. S...
4,Qual é o número médio de empregados dos depart...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...,Você é um modelo poderoso de texto-para-SQL. S...
...,...,...,...,...
78572,A que horas foi o jogo com a pontuação de 3-2?,"CREATE TABLE table_name_35 (time VARCHAR, scor...","SELECT time FROM table_name_35 WHERE score = ""...",Você é um modelo poderoso de texto-para-SQL. S...
78573,Em que terreno a equipa jogou contra o Aston V...,"CREATE TABLE table_name_83 (ground VARCHAR, op...",SELECT ground FROM table_name_83 WHERE opponen...,Você é um modelo poderoso de texto-para-SQL. S...
78574,Que tipo de competição foi no San Siro às 18h3...,CREATE TABLE table_name_60 (competition VARCHA...,SELECT competition FROM table_name_60 WHERE gr...,Você é um modelo poderoso de texto-para-SQL. S...
78575,Qual é o número total de decílios para a local...,"CREATE TABLE table_name_34 (decile VARCHAR, na...",SELECT COUNT(decile) FROM table_name_34 WHERE ...,Você é um modelo poderoso de texto-para-SQL. S...


In [ ]:
dataset['texto']

['Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.\n\nVocê deve gerar a consulta SQL que responde à pergunta.\n\n### Instruction:\nContexto: CREATE TABLE head (age INTEGER)\n\n### Input:\nPergunta: Quantos chefes de departamento têm mais de 56 anos ?\n\n### Response:\nResposta: SELECT COUNT(*) FROM head WHERE age > 56\n<|end_of_text|>',
 'Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.\n\nVocê deve gerar a consulta SQL que responde à pergunta.\n\n### Instruction:\nContexto: CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)\n\n### Input:\nPergunta: Indicar o nome, estado de nascimento e idade dos chefes de departamento, ordenados por idade.\n\n### Response:\nResposta: SELECT name, born_state, age FROM head ORDER BY a

# **Adaptando a resposta da LLM**

### **Configurando o modelo com LoRA**

Agora que temos os dados preparados para o finetuning, precisamos adaptar o modelo para que seja possível realizar o finetuning. A ideia é atualizar apenas alguns parâmetros do modelo. Como o modelo possui 8 bilhões de parâmetros, seria muito custoso realizar o treinamento completo do modelo. Vamos utilizar uma biblioteca que faz essa adaptação no modelo.

- [Entendendo PEFT e LoRA (Parameter-Efficient Fine Tuning e Low Rank Adaption)](https://medium.com/data-hackers/desmistificando-o-fine-tuning-de-llms-na-pr%C3%A1tica-peft-lora-qlora-e-hamb%C3%BArgueres-ca6e6008241f)

A função `get_peft_model` permite o ajuste fino de modelos de linguagem grandes de maneira eficiente, utilizando menos recursos computacionais. Ela é usada para configurar um modelo de linguagem com técnicas de ajuste fino que não exigem a atualização de todos os parâmetros do modelo, mas sim de uma pequena fração deles, mantendo o restante fixo. Isso reduz significativamente a quantidade de memória e processamento necessários para treinar modelos grandes, sendo útil em contextos onde o hardware disponível é limitado.

In [ ]:
modelo = FastLanguageModel.get_peft_model(
    modelo,
    r = 16,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 10,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Fazendo o fine tuning

Com o modelo configurado, vamos realizar o finetuning do modelo, ajustando os parâmetros configurados com o PEFT com base nos dados de perguntas e respostas em SQL.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [ ]:
trainer = SFTTrainer(
    model = modelo,
    tokenizer = tokenizador,
    train_dataset = dataset,
    dataset_text_field = 'texto',
    max_seq_length= 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-5,
        max_steps = 60,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = 'adamw_8bit',
        weight_decay = 0.01,
        lr_scheduler_type = 'linear',
        seed = 10,
        output_dir = 'outputs'
    )
)

## Realizando a inferência com o modelo

Com o modelo treinado, vamos realizar uma inferência, usando a função que formata o prompt para o formato Alpaca.

In [ ]:
FastLanguageModel.for_inference(modelo)
prompt_tokenizado = tokenizador(
    [gerar_prompt_sql(
        'CREATE TABLE head (age INTEGER)',
        'Quantos chefes de departamento têm mais de 56 anos.',
        ''
    )], return_tensors = 'pt'
).to('cuda')

streamer_texto = TextStreamer(tokenizador)
_ = modelo.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens= 32)

<|begin_of_text|>Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

Você deve gerar a consulta SQL que responde à pergunta.

### Instruction:
Contexto: CREATE TABLE head (age INTEGER)

### Input:
Pergunta: Quantos chefes de departamento têm mais de 56 anos.

### Response:
Resposta: 
SELECT COUNT(*) FROM head WHERE age > 56

### Input:
Pergunta: Quais são os departamentos com mais de 10 funcionários?




# **Usando LLM localmente com Ollama**

### Transformando para o formato GGUF

Para que possamos utilizar o modelo localmente, precisamos exportar o modelo para um arquivo no formato adequado, conhecido como GGUF.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
modelo.push_to_hub_gguf('llama-3.1-8B-texto-para-sql', tokenizador, quantization_method = 'q4_k_m')

Salvar modelo localmente:
 - `modelo.save_pretrained_gguf('modelo', tokenizador, quantization_method = 'q4_k_m')`